In [37]:
from logpy import run, var, fact
import logpy.assoccomm as la

In [38]:
# define mathematical operations
add = 'addition'
mul = 'multiplication'


In [39]:
# Both addition and multiplication are commutative operations(meaning the operands can be flipped without changing the result)
#Declare that these operations are commutatitve
#using the facts system
fact(la.commutative,mul)
fact(la.commutative,add)
fact(la.associative,mul)
fact(la.associative,add)

In [40]:
# Define some variables
a,b,c = var('a'), var('b'), var('c')

In [41]:
expression_orig = 3 * (-2) + (1 + 2 * 3) * (-1)

In [42]:
expression_origin = (add,(mul,3,-2),(mul,(add,1,(mul,2,3)),-1))
expression1 =(add,(mul,(add,1,(mul,2,a)),b),(mul,3,c))
expression2 =(add,(mul,c,3),(mul,b,(add,(mul,2,a),1)))
expression3= (add,(add,(mul,(mul,2,a),b),b),(mul,3,c))

In [ ]:
print(run(0,(a,b,c),la.eq_assoccomm(expression1,expression_orig)))
print(run(0,(a,b,c),la.eq_assoccomm(expression2,expression_orig)))
print(run(0,(a,b,c),la.eq_assoccomm(expression3,expression_orig)))

In [51]:
import itertools as it
import logpy.core as lc
from kanren import membero
from sympy.ntheory.generate import prime, isprime

In [48]:
# Check if the elements of x prime

def check_prime(x):
    if lc.isvar(x):
        return lc.condeseq([(lc.eq,x,p)] for p in map(prime,it.count(1)))
    else:
        return lc.success if isprime(x) else lc.fail

In [49]:
# Declare the variable
x = lc.var()

In [ ]:
# Check if an element in the list is a prime number
list_nums = (23,4,27,17,13,10,21,29,3,32,11,19)
print('\nList of primes in the list:')
print(set(lc.run(0,x,(membero,x,list_nums),(check_prime,x))))

In [157]:
import json
from logpy import Relation, facts, run, conde, var, eq

In [ ]:
with open('./files/familytree.json')as f:
    family_tree = json.loads(f.read())
family_tree

In [159]:
father = Relation()
mother =Relation()

In [160]:
for item in family_tree['father']:
    facts(father,(list(item.keys())[0],list(item.values())[0]))

In [161]:
for item in family_tree['mother']:
    facts(mother,(list(item.keys())[0],list(item.values())[0]))

In [162]:
# Check if 'x' is the parent of 'y'

def parent(x,y):
    return conde([father(x,y)], [mother(x,y)])

In [163]:
# Check if 'x' is the grandparent of 'y'
def grandparent(x,y):
    temp =var()
    return conde((parent(x,temp),parent(temp,y)))

In [164]:
#Check for sibling relationship between a and b
def sibling(x,y):
    temp = var()
    return conde((parent(temp,x),parent(temp,y)))

In [165]:
# Check if x is y's uncle
def uncle(x,y):
    temp = var()
    return conde((father(temp,x),grandparent(temp,y)))

In [166]:
x = var()

In [ ]:
# John's children
name = 'John'
output = run (0,x,father(name,x))
print(f"\nList of {name}'s children:")
for item in output:
    print(item)

In [ ]:
# William's mother
name = "William"

output = run(0,x, mother(x,name))[0]
print(f"\n {name}'s mother:\n {output}")

In [ ]:
# Adam's parents name = Adam

output = run(0, x, parent(x, name))
print(f"\nList of {name}'s parents:")
for item in output:
    print(item)

In [ ]:
# Wayne's parents name = 'Wayne'
output = run(0, x, grandparent(x, name))
print(f"\nList of {name}'s grandparents:")
for item in output:
    print(item)

In [ ]:
# Megan's grandparents
name = 'Megan'
output = run(0, x, grandparent(x, name))
print(f"\nList of {name}'s grandchildren:")
for item in output:
    print(item)

In [ ]:
# David's siblings
name = 'David'
output = run(0, x, sibling(x, name))
siblings = [x for x in output if x!= name]
print(f"\nList of {name}'s siblings:")
for item in siblings:
    print(item)

In [ ]:
# Tiffany's uncles
name = 'Tiffany'
name_father = run(0,x,father(x,name))[0]
output = run(0, x, uncle(x,name))
output = [x for x in output if x != name_father]
print(f"\nList of {name}'s uncles:")
for item in output:
    print(item)

In [ ]:
# All spouses
a,b,c = var(), var(), var()
output = run(0,(a,b), (father,a,c),(mother,b,c))
print("\nList of all spouses:")
for item in output:
    print(f"Husband: {item[0]}, <==> wife: {item[1]}")

In [247]:
from logpy import run, fact, eq, Relation,var
adjacent = Relation()
coastal = Relation()


In [248]:
file_coastal = './files/coastal-states.txt'
file_adjacent = './files/adjacent-states.txt'


In [249]:
# Read the file containing the coastal states
with open (file_coastal, 'r') as f:
    line = f.read()
    coastal_states = line.split(',')
# Add the info to the fact base
for state in coastal_states:
    fact(coastal,state)

In [ ]:
# Read the files containing the adjacent states
with open (file_adjacent, 'r') as f:
    adjlist = [line.strip().split(',') for line in f if line and line[0].isalpha()]
adjlist

In [251]:
# Add the info to the fact base
for L in adjlist:
    head, tail = L[0], L[1:]
    for state in tail:
        fact(adjacent,head,state)
        

In [252]:
# Initialize the variables
x =var()
y = var()

In [ ]:
# Is Nevada to Louisiana?
output = run (0,x, adjacent('NV','LA'))
print('\n Is Nevada adjacent to Lousina?:')
print("Yes" if len(output) else "No")

In [ ]:
# States adjacent to Oregon
output = run(0, x, adjacent('OR', x))
print('\nList of states adjacent to Oregon:')
for item in output:
    print(item)

In [ ]:
# States adjacent to Mississippi that are coastal
output = run(0, x, adjacent('MS', x), coastal(x))
print('\nList of coastal states adjacent to Mississippi:')
for item in output:
    print(item)

In [ ]:
# List of ‘n’ states that border a coastal state
n = 7
output = run(n, x, coastal(y), adjacent(x, y))
print('\nList of ‘ + str(n) + ‘ states that border a coastal state:')
for item in output:
    print(item)

In [ ]:
# List of states that adjacent to the two given states
output = run(0, x, adjacent('AK', x), adjacent('KY', x))
print('\nList of states that are adjacent to Arkansas and Kentucky:')
for item in output:
    print(item)

In [270]:
from logpy import *
from logpy.core import lall

In [271]:
# Declare the variable people
people = var()

In [272]:
# Define the rules
rules = lall(\
    # There are 4 people
    (eq, (var(), var(), var(), var()), people),
    # Steve’s car is blue
    (membero, ('Steve', var(), 'blue', var()), people),
    # Person who has a cat lives in Canada
    (membero, (var(), 'cat', var(), 'Canada'), people),
    # Matthew lives in USA
    (membero, ('Matthew', var(), var(), 'USA'), people),
    # Jack has a cat
    (membero, ('Jack', 'cat', var(), var()), people),
    # Alfred lives in Australia
    (membero, ('Alfred', var(), var(), 'Australia'), people),
    # Person who owns the dog lives in France
    (membero, (var(), 'dog', var(), 'France'), people),
    # Who has a rabbit?
    (membero, (var(), 'rabbit', var(), var()), people)
 
)



In [273]:
# Run the solver
solutions = run(0, people, rules)

In [274]:
# Extract the output
output = [house for house in solutions[0] if 'rabbit' in house][0][0]

In [ ]:
# Print the output
print('\n' + output + 'is the owner of the rabbit')
print('\nHere are all the details:')
attribs = ['Name', 'Pet', 'Color', 'Country']
print('\n' + '\t\t'.join(attribs))
print('=' * 57)
for item in solutions[0]:
    print('')
    print('\t\t'.join([str(x) for x in item]))
